In [1]:
from utils.pack import Pack
from utils.agent import Agent
from utils.metrics import ThoughtDiversity

## Single Agent

In [2]:
embedding_params = ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.7]
agent_hilbert = Agent('agent_hilbert_space',
                      'documents/HilbertSpaceMulti.pdf', 1, embedding_params, False)
agent_hilbert.chat_bot.one_question("What is hilbert space?")

🔥  Conjuring up agent_hilbert_space  🔥 

🧙 creating course  🧙

🔮 creating encoder  🔮 
🧩 chunks created 🧩
⚫
 🔗 embedding created 🔗
⚫


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🧚 creating chat_bot for  🧚

the path  🌈 being used for agent_hilbert_space is documents/HilbertSpaceMulti.pdf



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Hilbert space is a mathematical concept in quantum mechanics that represents the state of a quantum system. It is a complex vector space that has an inner product defined on it, allowing for the measurement of lengths and angles between vectors. In Hilbert space, vectors represent the possible states of a quantum system, and operators represent the observables or measurements that can be made on the system. The concept of Hilbert space is used to describe the behavior and evolution of quantum systems.'

## Pack

In [3]:
learning_to_act = "chroma_db/agent_ltoa"
system_neural_diversity = "chroma_db/agent_snd"
foundational_models = "chroma_db/agent_foundation"

embedding_params = [
    ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.9],
    ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.1],
    ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.5],
    ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.9],
    ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.1],
    ["facebook-dpr-ctx_encoder-multiset-base", 200, 25, 0.5]
]
# name, path, cot_type, new_bool
agent_specs = [
    ['agent_ltoa', learning_to_act, 0, True],
    ['agent_snd', system_neural_diversity, 0, True],
    ['agent_foundation', foundational_models, 0, True],
    ['agent_quant', 'documents/VisualizingQuantumCircuitProbability.pdf', 1, False],
    ['agent_norbert', 'documents/Norman-CognitiveEngineering.pdf', 1, False]
]

test_pack = Pack(agent_specs, embedding_params)

🔥  Conjuring up agent_ltoa  🔥 

🧙 creating course  🧙

🔮 creating encoder  🔮 
loading agent...
agent loaded

🧚 creating chat_bot for  🧚

the path  🌈 being used for agent_ltoa is chroma_db/agent_ltoa

🔥  Conjuring up agent_snd  🔥 

🧙 creating course  🧙

🔮 creating encoder  🔮 
loading agent...
agent loaded

🧚 creating chat_bot for  🧚

the path  🌈 being used for agent_snd is chroma_db/agent_snd

🔥  Conjuring up agent_foundation  🔥 

🧙 creating course  🧙

🔮 creating encoder  🔮 
loading agent...
agent loaded

🧚 creating chat_bot for  🧚

the path  🌈 being used for agent_foundation is chroma_db/agent_foundation

🔥  Conjuring up agent_quant  🔥 

🧙 creating course  🧙

🔮 creating encoder  🔮 
🧩 chunks created 🧩
⚫
 🔗 embedding created 🔗
⚫


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🧚 creating chat_bot for  🧚

the path  🌈 being used for agent_quant is documents/VisualizingQuantumCircuitProbability.pdf

🔥  Conjuring up agent_norbert  🔥 

🧙 creating course  🧙

🔮 creating encoder  🔮 


In [ ]:
test_pack.knn.visualize([0.178, -0.217], 3)

In [ ]:
test_pack.one_question("how to make better ai for better humans?")

## Metrics

In [ ]:
metrics = ThoughtDiversity(test_pack)

In [ ]:
# print(metrics.monte_carlo_sim("How can we better desing AI agents?"))